In [3]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())
# print(os.getenv("GOOGLE_API_KEY"))

True

In [6]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.tool_context import ToolContext


from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


Configure Retry Options
When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [20]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

Long-Running Operations (Human-in-the-Loop)
So far, all tools execute and return immediately:

User asks → Agent calls tool → Tool returns result → Agent responds

But what if your tools are long-running or you need human approval before completing an action?

Example: A shipping agent should ask for approval before placing a large order.

User asks → Agent calls tool → Tool PAUSES and asks human → Human approves → Tool completes → Agent responds

This is called a Long-Running Operation (LRO) - the tool needs to pause, wait for external input (human approval), then resume.

When to use Long-Running Operations:

💰 Financial transactions requiring approval (transfers, purchases)

🗑️ Bulk operations (delete 1000 records - confirm first!)

📋 Compliance checkpoints (regulatory approval needed)

💸 High-cost actions (spin up 50 servers - are you sure?)

⚠️ Irreversible operations (permanently delete account)

What We're Building Today

Let's build a shipping coordinator agent with one tool that:

Auto-approves small orders (≤5 containers)


Pauses and asks for approval on large orders (>5 containers)

Completes or cancels based on the approval decision

This demonstrates the core long-running operation pattern: pause → wait for human input → resume.

The Shipping Tool with Approval Logic

Here's the complete function.

The ToolContext Parameter

Notice the function signature includes tool_context: ToolContext. ADK automatically provides this object when your tool runs. It gives you two key capabilities:

Request approval: Call tool_context.request_confirmation()

Check approval status: Read tool_context.tool_confirmation

In [21]:
LARGE_ORDER_THRESHOLD = 5


def place_shipping_order(
    num_containers: int, destination: str, tool_context: ToolContext
) -> dict:
    """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).

    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination

    Returns:
        Dictionary with order status
    """

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Small orders (≤5 containers) auto-approve
    if num_containers <= LARGE_ORDER_THRESHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order auto-approved: {num_containers} containers to {destination}",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
            payload={"num_containers": num_containers, "destination": destination},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved: {num_containers} containers to {destination}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }


print("✅ Long-running functions created!")

✅ Long-running functions created!


How the Three Scenarios Work

The tool handles three scenarios by checking tool_context.tool_confirmation:

Scenario 1: Small order (≤5 containers): Returns immediately with auto-approved status.

tool_context.tool_confirmation is never checked

Scenario 2: Large order - FIRST CALL

Tool detects it's a first call: if not tool_context.tool_confirmation:

Calls request_confirmation() to request human approval

Returns {'status': 'pending', ...} immediately

ADK automatically creates adk_request_confirmation event

Agent execution pauses - waiting for human decision

Scenario 3: Large order - RESUMED CALL

Tool detects it's resuming: if not tool_context.tool_confirmation: is now False

Checks human decision: tool_context.tool_confirmation.confirmed

If True → Returns approved status

If False → Returns rejected status

Key insight: Between the two calls, your workflow code (in Section 4) must detect the adk_request_confirmation event and resume with the approval decision.

Create the Agent, App and Runner

# Step 1: Create the agent

## Add the tool to the Agent. The tool decides internally when to request approval based on the order size.

In [22]:
# Create shipping agent with pausable tool
shipping_agent = LlmAgent(
    name="shipping_agent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""You are a shipping coordinator assistant.
  
  When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
  """,
    tools=[FunctionTool(func=place_shipping_order)],
)

print("✅ Shipping Agent created!")

✅ Shipping Agent created!


Step 2: Wrap in resumable App

The problem: A regular LlmAgent is stateless - each call is independent with no memory of previous interactions. If a tool requests approval, the agent can't remember what it was doing.

The solution: Wrap your agent in an App with resumability enabled. The App adds a persistence layer that saves and restores state.

What gets saved when a tool pauses:

All conversation messages so far

Which tool was called (place_shipping_order)

Tool parameters (10 containers, Rotterdam)

Where exactly it paused (waiting for approval)

When you resume, the App loads this saved state so the agent continues exactly where it left off - as if no time passed.

In [23]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
shipping_app = App(
    name="shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


C:\Users\mjkat\AppData\Local\Temp\ipykernel_25364\3673777575.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


Step 3: Create Session and Runner with the App

Pass app=shipping_app instead of agent=... so the runner knows about resumability.

In [24]:
session_service = InMemorySessionService()

# Create runner with the resumable app
shipping_runner = Runner(
    app=shipping_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

✅ Runner created!


✅ Recap: Your pausable shipping agent is now complete!

You've created:

✅ A tool that can pause for approval (place_shipping_order)

✅ An agent that uses this tool (shipping_agent)

✅ A resumable app that saves state (shipping_app)

✅ A runner that can handle pause/resume (shipping_runner)

Next step: Build the workflow code and test that our Agent detects pauses and handles approvals.

# Building the Workflow

The Critical Part - Handling Events in Your Workflow

The agent won't automatically handle pause/resume. Every long-running operation workflow requires you to:


Detect the pause: Check if events contain adk_request_confirmation

Get human decision: In production, show UI and wait for user click. Here, we simulate it.

Resume the agent: Send the decision back with the saved invocation_id

Understand Key Technical Concepts

👉 events - ADK creates events as the agent executes. Tool calls, model responses, function results - all become events

👉 adk_request_confirmation event - This event is special - it signals "pause here!"

Automatically created by ADK when your tool calls request_confirmation()

Contains the invocation_id

Your workflow must detect this event to know the agent paused

👉 invocation_id - Every call to run_async() gets a unique invocation_id (like "abc123")

When a tool pauses, you save this ID

When resuming, pass the same ID so ADK knows which execution to continue

Without it, ADK would start a NEW execution instead of resuming the paused one

check_for_approval() - Detects if the agent paused

Loops through all events and looks for the special adk_request_confirmation event

Returns approval_id (identifies this specific request) and invocation_id (identifies which execution to resume)

Returns None if no pause detected

In [25]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

print_agent_response() - Displays agent text

Simple helper to extract and print text from events

In [26]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

create_approval_response() - Formats the human decision

Takes the approval info and boolean decision (True/False) from the human

Creates a FunctionResponse that ADK understands

Wraps it in a Content object to send back to the agent

In [27]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


The Workflow Function - Let's tie it all together!

The run_shipping_workflow() function orchestrates the entire approval flow.

Look for the code explanation in the cell below.

In [28]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
    """Runs a shipping workflow with approval handling.

    Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns the special `adk_request_confirmation` event
    async for event in shipping_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Workflow function ready")

✅ Workflow function ready


Code breakdown

Step 1: Send initial request to the Agent

Call run_async() to start agent execution

Collect all events in a list for inspection

Step 2: Detect Pause

Call check_for_approval(events) to look for the special event: adk_request_confirmation

Returns approval info (with invocation_id) if the special event is present; None if completed

Step 3: Resume execution

PATH A:

If the approval info is present, at this point the Agent pauses for human input.
Once the Human input is available, call the agent again using run_async() and pass in the Human input.

Critical: Same invocation_id (tells ADK to RESUME, not restart)

Display agent's final response after resuming

PATH B:

If the approval info is not present, then approval is not needed and the agent completes execution.

# Demo: Testing the Workflow

Now, let's run our demos. Notice how much cleaner and easier to read they are. All the complex logic for pausing and resuming is now hidden away in our 
run_workflow helper function, allowing us to focus on the tasks we want the agent to perform.

Note: You may see warnings like Warning: there are non-text parts in the response: ['function_call'] - this is normal and can be ignored. It just means the agent is calling tools in addition to generating text.

In [29]:
# Demo 1: It's a small order. Agent receives auto-approved status from tool
await run_shipping_workflow("Ship 3 containers to Singapore")

# Demo 2: Workflow simulates human decision: APPROVE ✅
await run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=True)

# Demo 3: Workflow simulates human decision: REJECT ❌
await run_shipping_workflow("Ship 8 containers to Los Angeles", auto_approve=False)


User > Ship 3 containers to Singapore



Agent > Your order for 3 containers to Singapore has been approved with order ID ORD-3-AUTO.


User > Ship 10 containers to Rotterdam

⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅

Agent > Order status: approved
Order ID: ORD-10-HUMAN
10 containers to Rotterdam


User > Ship 8 containers to Los Angeles

⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > Order status: rejected
Number of containers: 8
Destination: Los Angeles



---

## 📊 Summary - Key Patterns for Advanced Tools

In this notebook, you implemented two powerful, production-ready patterns for extending your agent's capabilities beyond simple functions.

| Pattern | When to Use It | Key ADK Components |
| :--- | :--- | :--- |
| 
| **Long-Running Operations** | You need to **pause a workflow** to wait for an external event, most commonly for **human-in-the-loop** approvals or long background tasks or for compliance/security checkpoints. | `ToolContext`, `request_confirmation`, `App`, `ResumabilityConfig` |

### 🚀 Production Ready Concepts

You now understand how to build agents that:
- 🌐 **Scale**: Leverage community tools instead of building everything
- ⏳ **Handle Time**: Manage operations that span minutes, hours, or days  
- 🔒 **Ensure Compliance**: Add human oversight to critical operations
- 🔄 **Maintain State**: Resume conversations exactly where they paused

